<a href="https://colab.research.google.com/github/SantiagoGomezfpv/hyperparameter/blob/main/Perceptr%C3%B3n_multicapa_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optuna Perceptrón multicapa


Este notebook de Colab está diseñado para entrenar y optimizar un modelo de red neuronal multicapa (MLP) utilizando el conjunto de datos **MNIST**, que contiene imágenes de dígitos escritos a mano. Usaremos la biblioteca **Optuna** para encontrar la mejor configuración de hiperparámetros para nuestro modelo.

## Primero, realizamos las instalaciones necesarias:

In [2]:
!pip install optuna

## Segundo, importamos las bibliotecas necesarias:

* `optuna` se utiliza para la optimización de hiperparámetros.
* `torch` y `torchvision` son bibliotecas de PyTorch para construir y entrenar modelos de aprendizaje profundo.
* `nn` y `nn.functional` contienen herramientas para crear redes neuronales.
* `datasets` y `transforms` se utilizan para manejar los datos del conjunto de datos MNIST.
* `DataLoader` nos ayuda a cargar los datos en mini-lotes.

In [3]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Tercero, cargar el Conjunto de Datos MNIST
El conjunto de datos MNIST se compone de imágenes de dígitos escritos a mano. Convertimos las imágenes a tensores y normalizamos los valores de píxeles.

* `transforms.Compose` crea una lista de transformaciones a aplicar a las imágenes.
* `transforms.ToTensor` convierte las imágenes en tensores.
* `transforms.Normalize` normaliza los píxeles de las imágenes.

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),                # Convertir las imágenes a tensores
    transforms.Normalize((0.5,), (0.5,))  # Normalizar los valores de píxeles
])

## Luego, división del Conjunto de Datos
Descargamos el conjunto de datos MNIST y lo dividimos en conjuntos de entrenamiento y validación. También cargamos el conjunto de prueba.

* `datasets.MNIST` descarga el conjunto de datos MNIST.
* `random_split` divide el conjunto de datos de entrenamiento en dos subconjuntos.
* `DataLoader` carga los datos en mini-lotes para entrenamiento y validación.



In [5]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

trainset, valset = torch.utils.data.random_split(trainset, [54000, 6000])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 12696878.02it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 365972.18it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 2752332.87it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2147748.45it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Definición del Modelo MLP
Creamos una clase para nuestra red neuronal multicapa (MLP).
* `nn.Module` es la clase base para todas las redes neuronales en PyTorch.
* `nn.Linear` crea una capa lineal.
* `nn.ReLU` aplica la función de activación ReLU.
* `nn.Sequential` encadena las capas de la red neuronal.


In [6]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_units_list, output_size):
        super(MLP, self).__init__()
        layers = []
        in_features = input_size

        for hidden_units in hidden_units_list:
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(nn.ReLU())
            in_features = hidden_units

        layers.append(nn.Linear(in_features, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

## Definición de la Función Objetivo
Definimos una función objetivo para Optuna que optimizará los hiperparámetros del modelo.

* `trial.suggest_int` permite a Optuna probar diferentes valores de hiperparámetros.
* `CrossEntropyLoss` se utiliza para calcular la pérdida del modelo.
* `Adam` es un optimizador de gradiente descendente.
* `model.train()` y `model.eval()` cambian el modo del modelo entre entrenamiento y evaluación.
* `torch.no_grad()` desactiva el cálculo de gradientes para ahorrar memoria durante la evaluación.

Optuna está optimizando los siguientes hiperparámetros de la red neuronal:

* `hidden_layers:` Define cuántas capas ocultas tendrá la red neuronal. Optuna prueba diferentes cantidades de capas ocultas dentro del rango especificado (1 a 3).

* `hidden_units_list:` Para cada capa oculta, define cuántas unidades (neurona) tendrá esa capa. Optuna prueba diferentes cantidades de unidades en cada capa dentro del rango especificado (32 a 256).

In [7]:
def objective(trial):
    hidden_layers = trial.suggest_int('hidden_layers', 1, 3)
    hidden_units_list = [trial.suggest_int(f'hidden_units_{i}', 32, 256) for i in range(hidden_layers)]

    model = MLP(28*28, hidden_units_list, 10).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(5):
        for images, labels in trainloader:
            images = images.view(-1, 28*28).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valloader:
            images = images.view(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

## Ejecución de la Optimización
Finalmente, ejecutamos la optimización de hiperparámetros con Optuna.

* `torch.device` selecciona el dispositivo (CPU o GPU) para ejecutar el modelo.
* `optuna.create_study` crea un estudio de optimización.
* `study.optimize` ejecuta la optimización de la función objetivo durante un número definido de pruebas.

El mejor valor de precisión y los mejores hiperparámetros se imprimen al final.

In [8]:
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-07-10 16:45:31,929] A new study created in memory with name: no-name-2aa052e3-6071-48a4-8482-b41f141b65c2
[I 2024-07-10 16:46:33,131] Trial 0 finished with value: 0.9695 and parameters: {'hidden_layers': 1, 'hidden_units_0': 171}. Best is trial 0 with value: 0.9695.
[I 2024-07-10 16:47:31,551] Trial 1 finished with value: 0.972 and parameters: {'hidden_layers': 1, 'hidden_units_0': 157}. Best is trial 1 with value: 0.972.
[I 2024-07-10 16:48:31,355] Trial 2 finished with value: 0.968 and parameters: {'hidden_layers': 3, 'hidden_units_0': 214, 'hidden_units_1': 148, 'hidden_units_2': 49}. Best is trial 1 with value: 0.972.
[I 2024-07-10 16:49:30,554] Trial 3 finished with value: 0.9718333333333333 and parameters: {'hidden_layers': 1, 'hidden_units_0': 249}. Best is trial 1 with value: 0.972.
[I 2024-07-10 16:50:32,662] Trial 4 finished with value: 0.9696666666666667 and parameters: {'hidden_layers': 2, 'hidden_units_0': 138, 'hidden_units_1': 245}. Best is trial 1 with value: 0.

Number of finished trials:  50
Best trial:
  Value:  0.9751666666666666
  Params: 
    hidden_layers: 3
    hidden_units_0: 153
    hidden_units_1: 141
    hidden_units_2: 126


## Visualización de los Resultados
Optuna proporciona herramientas de visualización para entender mejor los resultados de la optimización. Agregamos visualizaciones para ver la historia de la optimización, las coordenadas paralelas y la importancia de los parámetros.

* `plot_optimization_history`: Muestra cómo ha cambiado el valor objetivo a lo largo de los ensayos.
* `plot_parallel_coordinate`: Muestra las relaciones entre los parámetros y el valor objetivo.
* `plot_param_importances`: Muestra la importancia relativa de cada parámetro.

In [9]:
import matplotlib.pyplot as plt
best_params = study.best_params

optuna.visualization.plot_optimization_history(study)

In [10]:
optuna.visualization.plot_parallel_coordinate(study)

In [11]:
optuna.visualization.plot_param_importances(study)